# Visual comparison of parameters for images in SAR and geographic coordinates

- This is only to check the outputs for geocoded images
- Numerical comparison is done in a script with C outputs using reference data.

In [ ]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
from pathlib import Path
import xarray as xr
from polsarpro.io import open_netcdf_beam
from polsarpro.decompositions import freeman
import matplotlib.pyplot as plt
# optional import for progress bar
from dask.diagnostics import ProgressBar

# change to your data paths
# original dataset
input_alos_slc = Path("/data/psp/test_files/SAN_FRANCISCO_ALOS1_slc.nc")
input_alos_geo = Path("/data/psp/test_files/SAN_FRANCISCO_ALOS1_geocoded_T3_7_look_az.nc")

# input files from C
input_test_dir = Path("/data/psp/test_files/input/h_a_alpha_decomposition/")

# output files from C
output_test_dir = Path("/data/psp/res/h_a_alpha_c")

In [ ]:
slc = open_netcdf_beam(input_alos_slc)
geo = open_netcdf_beam(input_alos_geo)

In [ ]:
# let's use persist (retains dask arrays) instead of compute (produces numpy arrays)
with ProgressBar():
    res_slc = freeman(slc, boxcar_size=[7 ,7]).persist()
with ProgressBar():
    # data has been multilooked in azimuth so we need less filtering
    res_geo = freeman(geo, boxcar_size=[3 ,3]).persist()

In [ ]:
for var in res_geo.data_vars:
    plt.figure(figsize=(10, 10))
    plt.suptitle(var)
    plt.subplot(121)
    plt.imshow(res_slc[var][::8], interpolation="none", vmax=1)
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.subplot(122)
    plt.imshow(res_geo[var], interpolation="none", vmax=1)
    plt.colorbar(fraction=0.046, pad=0.04)